In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import DateType, StringType
from datetime import datetime
from pyspark.sql.functions import date_format
import os
import sys
import psycopg2
import pandas as pd
from os import listdir
from os.path import isfile, join
import re
import matplotlib.pyplot as plot

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName('Covid-19').getOrCreate()
df_covid = spark.read.json("test-2022-05-04-09-41-41")
df_covid_deaths = df_covid.select(df_covid.date, df_covid.death).filter(func.col("death").isNotNull())
df_covid_deaths = df_covid_deaths.withColumn("date", df_covid_deaths["date"].cast(StringType()))
df_covid_deaths.show()

+--------+------+
|    date| death|
+--------+------+
|20210307|515151|
|20210306|514309|
|20210305|512629|
|20210304|510408|
|20210303|508665|
|20210302|506216|
|20210301|504488|
|20210228|503247|
|20210227|502196|
|20210226|500349|
|20210225|498208|
|20210224|495070|
|20210223|492623|
|20210222|490382|
|20210221|489147|
|20210220|487860|
|20210219|485700|
|20210218|483223|
|20210217|480607|
|20210216|478259|
+--------+------+
only showing top 20 rows



In [4]:
my_udf =  func.udf (lambda x: datetime.strptime(x, '%Y%m%d'), DateType())
df = df_covid_deaths.withColumn("date", date_format(my_udf(func.col('date')), 'MM-dd-yyy'))
df.show()

+----------+------+
|      date| death|
+----------+------+
|03-07-2021|515151|
|03-06-2021|514309|
|03-05-2021|512629|
|03-04-2021|510408|
|03-03-2021|508665|
|03-02-2021|506216|
|03-01-2021|504488|
|02-28-2021|503247|
|02-27-2021|502196|
|02-26-2021|500349|
|02-25-2021|498208|
|02-24-2021|495070|
|02-23-2021|492623|
|02-22-2021|490382|
|02-21-2021|489147|
|02-20-2021|487860|
|02-19-2021|485700|
|02-18-2021|483223|
|02-17-2021|480607|
|02-16-2021|478259|
+----------+------+
only showing top 20 rows



In [7]:
df = df_covid_deaths.withColumn("date_month_year", date_format(my_udf(func.col('date')), 'MM-dd-yyy'))
df.write.mode("overwrite").csv(path="ClimateChangeModified/")

Py4JJavaError: An error occurred while calling o75.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:287)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:6

In [37]:
driver = "org.postgresql.Driver"
url = ""
table = "deathtodate"
user = ""
password = ""
db_name = ""

files = [f for f in listdir("ClimateChangeModified") if isfile(join("ClimateChangeModified", f))]

for f in files:
    if re.search(".csv$", f):
        try:
            conn = psycopg2.connect(f"host='{url}' port='5432' dbname='{db_name}' user='{user}' password='{password}'")
            cur = conn.cursor()
            f = open(r'ClimateChangeModified/' + f, 'r')
            cur.copy_from(f, table, sep=',', columns=["date", "death"])
            conn.commit()
            conn.close()
            print("Success")
        except:
            print("Error With Connecting Server")

In [ ]:
files = [f for f in listdir("ClimateChangeModified") if isfile(join("ClimateChangeModified", f))]

for f in files:
    if re.search(".csv$", f):
        dataFrame = pd.read_csv("ClimateChangeModified/" + f, names=["date", "death"])


In [ ]:
dataFrame['date']= pd.to_datetime(dataFrame['date'])
dataFrame.plot(x="date", y="death", kind="area")